In [16]:
import torch
from torch.utils.tensorboard import SummaryWriter

In [2]:
torch.randn(2, 3, 20)

tensor([[[-0.3056, -0.2502, -0.1151, -0.1344, -1.4462, -1.2061, -0.3810,
           0.3697, -0.3140,  0.7101,  0.5875,  0.9239,  0.8547, -0.4835,
          -0.5631,  0.4409,  0.0294,  1.3858, -3.1313, -1.5738],
         [ 0.3428,  1.1288, -0.3470, -1.9055, -0.8913, -0.4335,  2.3119,
          -1.4110, -0.4648,  0.2264, -1.7857,  0.9380,  1.2593,  0.5636,
          -0.8744, -0.0256,  0.3173, -0.6592,  0.7471,  0.3569],
         [ 0.6094,  0.7370,  1.2534, -0.1765, -0.0745, -0.3292,  1.6391,
          -0.4543,  0.6439,  0.8137,  0.8034, -0.0803,  0.1001, -1.1750,
           1.4378, -0.5102, -0.7792, -1.4679, -0.2817, -0.4222]],

        [[ 0.0133, -1.1547, -1.4468, -0.4599, -0.6032, -1.6393, -1.7404,
           0.4116,  0.7275,  0.5113, -0.4436,  0.7263,  0.3210, -2.7261,
           2.0588, -0.9359, -0.6770, -1.1320, -1.4527, -2.7223],
         [-2.2226,  1.4286, -0.7130, -0.9019, -0.9149, -1.9909,  1.3421,
           2.0358,  0.4548,  1.0056,  0.1198, -0.4054,  0.1768, -0.4993,
        

# RNN

In [32]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
# input_size = 784 # 28x28
num_classes = 10
num_epochs = 2
batch_size = 100
initial_lr = 0.001

input_size = 28
sequence_length = 28
hidden_size = 128
num_layers = 2

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./dataset',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./dataset',
                                          train=False,
                                          transform=transforms.ToTensor(),
                                          download=True)

# Data loader  image,targets的数据形式
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # 这一部分的书写主要对应在观察RNN的parameters，需要input_size, hidden_size, num_layers和batch_first
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # 这里的batch_first=True 影响的是后来输入input的x 而不是这里的RNN model
        self.linear = nn.Linear(in_features=hidden_size, out_features=10)

    def forward(self, x):
        # x作为input在batch_first=True的时候应该输入的形状为N，L，Hin
        # 同时同时作为输入的还有h_0, (1*num_layers, N, Hout)
        # 这一部分的书写主要对应于inputs，x要满足input形状条件，同时提供符合形状条件的h0
        h_0 = torch.zeros(size=(self.num_layers, x.shape[0], self.hidden_size)).to(device)

        # 这一部分的书写主要对应于output部分，观察output输出几部分，分别是什么，对于模型，我们需要的是哪一部分，找到筛选对应的部分
        output, _ = self.rnn(x, h_0)   # 这里只选择要output，不要每一次计算生成的新的hidden_size

        # output的形状是下图的（N，L，D*Hout），最后一个纬度的Hout就是hidden_size
        output_last = output[:, -1, :]
        # output形状是
        final = self.linear(output_last)
        return final

## 模型编写部分

![](img_6.png)

## Inputs书写部分

![](img_4.png)

## Output选择筛选部分

![](img_5.png)

In [44]:
# Model
model = RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)

# SummaryWriter
writer = SummaryWriter(log_dir='RNN_Logs')

# Optimizer and Loss func
optimizer = torch.optim.Adam(params=model.parameters(), lr=initial_lr)
loss_fn = nn.CrossEntropyLoss()
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    # Train
    model.train()
    for i, (imgs, labels) in enumerate(train_loader):
        writer.add_images(tag='Orginal', img_tensor=imgs, global_step=i)

        # Reshape
        # print(imgs.shape) # Size([100, 1, 28, 28])
        # 根据RNN模型要求，我们需要的是（N，L，H），不需要Channel
        imgs = imgs.reshape(-1, sequence_length, input_size).to(device)

        # imgs = imgs.reshape(-1, 2, 28, 28).to(device)
        # writer.add_images(tag='After', img_tensor=imgs, global_step=i)
        # dataformats='NCHW'，这里会默认有Channel，同时对于图片来说，单通道的图片无法转化成RGB---3通道图片，也无法变成2个通道的照片，无法显示
        # 因而上面writer.add_images行是无法正常运行的 会报错
        labels = labels.to(device)
        outputs = model(imgs)

        # Loss&Optimizer
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        writer.add_scalar(tag='RNN_Train_Loss', scalar_value=loss, global_step=i+1)

        if (i+1) % 100 == 0:
            print(f'RNN: Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss}')

RNN: Epoch [1/2], Step [100/600], Loss: 1.0202053785324097
RNN: Epoch [1/2], Step [200/600], Loss: 0.9119145274162292
RNN: Epoch [1/2], Step [300/600], Loss: 0.769856870174408
RNN: Epoch [1/2], Step [400/600], Loss: 0.43682318925857544
RNN: Epoch [1/2], Step [500/600], Loss: 0.3267645239830017
RNN: Epoch [1/2], Step [600/600], Loss: 0.3402518332004547
RNN: Epoch [2/2], Step [100/600], Loss: 0.3188333809375763
RNN: Epoch [2/2], Step [200/600], Loss: 0.2547306418418884
RNN: Epoch [2/2], Step [300/600], Loss: 0.22514106333255768
RNN: Epoch [2/2], Step [400/600], Loss: 0.2613934278488159
RNN: Epoch [2/2], Step [500/600], Loss: 0.24806368350982666
RNN: Epoch [2/2], Step [600/600], Loss: 0.19717146456241608


In [45]:
# Test
RNN_Total_Loss = 0
test_samples = 0
test_correct = 0
model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        test_output = model(imgs)
        loss = loss_fn(test_output, labels)
        # max
        _, result_indices = torch.max(test_output, dim=1)
        RNN_Total_Loss += loss
        test_samples += len(labels)
        test_correct += (result_indices == labels).sum().item()
    print(f'Accuracy of the RNN on the 10000 test images: {test_correct*100/test_samples:.4f} %')
    print(f'RNN TOTAL LOSS: {RNN_Total_Loss}')

Accuracy of the RNN on the 10000 test images: 94.9800 %
RNN TOTAL LOSS: 18.07237434387207


# LSTM

In [52]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
# input_size = 784 # 28x28
num_classes = 10
num_epochs = 2
batch_size = 100
initial_lr = 0.001

input_size = 28
sequence_length = 28
hidden_size = 128
num_layers = 2

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./dataset',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./dataset',
                                          train=False,
                                          transform=transforms.ToTensor(),
                                          download=True)

# Data loader  image,targets的数据形式
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # 这一部分的书写主要对应在观察RNN的parameters，需要input_size, hidden_size, num_layers和batch_first
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # 这里的batch_first=True 影响的是后来输入input的x 而不是这里的RNN model
        self.linear = nn.Linear(in_features=hidden_size, out_features=10)

    def forward(self, x):
        # x作为input在batch_first=True的时候应该输入的形状为N，L，Hin
        # 同时同时作为输入的还有h_0, (1*num_layers, N, Hout)
        # 这一部分的书写主要对应于inputs，x要满足input形状条件，同时提供符合形状条件的h0
        h_0 = torch.zeros(size=(self.num_layers, x.shape[0], self.hidden_size)).to(device)
        c_0 = torch.zeros(size=(self.num_layers, x.shape[0], self.hidden_size)).to(device)

        # 这一部分的书写主要对应于output部分，观察output输出几部分，分别是什么，对于模型，我们需要的是哪一部分，找到筛选对应的部分
        output, _= self.lstm(x, (h_0, c_0))   # 这里只选择要output，不要每一次计算生成的新的hidden_size
        # 注意LSTM的是要求(h_0, c_0)要合在一起；同时output也是output, (h_n, c_n)的形式

        # output的形状是下图的（N，L，D*Hout），最后一个纬度的Hout就是hidden_size
        # Decode the hidden state of the last time step
        output_last = output[:, -1, :]
        # output形状是
        final = self.linear(output_last)
        return final

In [53]:
# Model
model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)

# SummaryWriter
writer = SummaryWriter(log_dir='LSTM_Logs')

# Optimizer and Loss func
optimizer = torch.optim.Adam(params=model.parameters(), lr=initial_lr)
loss_fn = nn.CrossEntropyLoss()
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    # Train
    model.train()
    for i, (imgs, labels) in enumerate(train_loader):
        writer.add_images(tag='Orginal', img_tensor=imgs, global_step=i)

        # Reshape
        # print(imgs.shape) # Size([100, 1, 28, 28])
        # 根据RNN模型要求，我们需要的是（N，L，H），不需要Channel
        imgs = imgs.reshape(-1, sequence_length, input_size).to(device)

        # imgs = imgs.reshape(-1, 2, 28, 28).to(device)
        # writer.add_images(tag='After', img_tensor=imgs, global_step=i)
        # dataformats='NCHW'，这里会默认有Channel，同时对于图片来说，单通道的图片无法转化成RGB---3通道图片，也无法变成2个通道的照片，无法显示
        # 因而上面writer.add_images行是无法正常运行的 会报错
        labels = labels.to(device)
        outputs = model(imgs)

        # Loss&Optimizer
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        writer.add_scalar(tag='RNN_Train_Loss', scalar_value=loss, global_step=i+1)

        if (i+1) % 100 == 0:
            print(f'RNN: Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss}')

RNN: Epoch [1/2], Step [100/600], Loss: 0.5816653370857239
RNN: Epoch [1/2], Step [200/600], Loss: 0.39331120252609253
RNN: Epoch [1/2], Step [300/600], Loss: 0.3043815493583679
RNN: Epoch [1/2], Step [400/600], Loss: 0.10253220796585083
RNN: Epoch [1/2], Step [500/600], Loss: 0.06371147185564041
RNN: Epoch [1/2], Step [600/600], Loss: 0.16674530506134033
RNN: Epoch [2/2], Step [100/600], Loss: 0.1519927829504013
RNN: Epoch [2/2], Step [200/600], Loss: 0.12093736976385117
RNN: Epoch [2/2], Step [300/600], Loss: 0.08417823910713196
RNN: Epoch [2/2], Step [400/600], Loss: 0.09751875698566437
RNN: Epoch [2/2], Step [500/600], Loss: 0.08690796792507172
RNN: Epoch [2/2], Step [600/600], Loss: 0.20972710847854614


In [54]:
# Test
LSTM_Total_Loss = 0
test_samples = 0
test_correct = 0
model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        test_output = model(imgs)
        loss = loss_fn(test_output, labels)
        # max
        _, result_indices = torch.max(test_output, dim=1)
        LSTM_Total_Loss += loss
        test_samples += len(labels)
        test_correct += (result_indices == labels).sum().item()
    print(f'Accuracy of the LSTM on the 10000 test images: {test_correct*100/test_samples:.4f} %')
    print(f'RNN TOTAL LOSS: {LSTM_Total_Loss}')

Accuracy of the LSTM on the 10000 test images: 97.3400 %
RNN TOTAL LOSS: 8.772577285766602


In [7]:
input1 = torch.randn(2).unsqueeze(0)
print(input1.shape)
input1

torch.Size([1, 2])


tensor([[2.5516, 0.0907]])

In [8]:
input2 = torch.randn(2)
print(input2.shape)
input2

torch.Size([2])


tensor([0.9617, 0.9042])